In [2]:
import pickle
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import describe
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from time import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import TruncatedSVD
import scipy
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer
#from dask_searchcv import GridSearchCV
from scipy.stats.distributions import norm
import math
%matplotlib inline

In [3]:
data = pickle.load(open("processed_data.pkl", "rb"))

In [4]:
possible_tags = pickle.load(open("possible_tags.pkl", "rb"))

In [5]:
base = data["base"]
labels = data["tags"]
use = data["use"]
description = data["description"]

In [6]:
base_sparse = scipy.sparse.csr_matrix(base)

In [7]:
combined = scipy.sparse.csr_matrix(scipy.sparse.hstack((base_sparse, use, description)))

In [8]:
num_datapoints = combined.shape[0]
test_cutoff = int(num_datapoints * 0.8)
val_cutoff = int(test_cutoff * 0.8)

combined_train, combined_test = combined[:test_cutoff], combined[test_cutoff:]
combined_train, combined_val = combined_train[:val_cutoff], combined_train[val_cutoff:]

labels_train, labels_test = labels[:test_cutoff], labels[test_cutoff:]
labels_train, labels_val = labels_train[:val_cutoff], labels_train[val_cutoff:]

In [9]:
print(len(labels_val))

12313


In [10]:
# 99.9 - 0.99955828617
# 99.5 - 0.9962458641
# 99 - 0.99175748541
# 98 - 0.98247287501
# 97 - 0.97301314912
# 96 - 0.96346130399
# 95 - 0.95384964725
def max_recall_score(labels, predictions, p_threshold=0.99175748541):
    predictions = np.array(predictions)
    total = 0.
    amounts = 0.
    for tag in range(25):
        precision, recall, thresholds = precision_recall_curve(labels[:,tag], predictions[tag,:,1])
        amount = sum(labels[:,tag])
        amounts += amount
        for i, p in enumerate(precision):
            if p > p_threshold:
                print("Recall for ", possible_tags[tag], recall[i])
                print("Threshold:", thresholds[i-1])
                total += recall[i] * amount
                break
    return total/amounts

In [11]:
# param_grid = {
#         "n_estimators": [640, 320, 160, 80, 40, 20, 10],
#    "max_features": [None, "sqrt", "log2"],
#         "max_depth": [None, 64, 32, 16, 8, 4, 2],
#         "min_samples_split": [2, 4, 8, 16, 32, 64, 128],
# }
    
clf = RandomForestClassifier(class_weight="balanced", verbose=3, n_estimators=80, n_jobs=-1, min_samples_split=16,
                            max_features=0.5, min_impurity_split=1e-06)
# clf = GridSearchCV(clf, param_grid, n_jobs=-1, verbose=3,
#                    scoring=make_scorer(max_recall_score, needs_proba=True), refit=True)
# n_estimators=80: 0.226512852808

In [12]:
clf.fit(combined_train, labels_train)
print(max_recall_score(labels_val, clf.predict_proba(combined_val)))

building tree 1 of 80building tree 3 of 80
building tree 5 of 80building tree 2 of 80building tree 8 of 80building tree 4 of 80building tree 6 of 80building tree 7 of 80






building tree 9 of 80
building tree 10 of 80
building tree 11 of 80
building tree 12 of 80
building tree 13 of 80
building tree 14 of 80
building tree 15 of 80
building tree 16 of 80
building tree 17 of 80
building tree 18 of 80
building tree 19 of 80
building tree 20 of 80
building tree 21 of 80
building tree 22 of 80
building tree 23 of 80


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 11.1min


building tree 24 of 80
building tree 25 of 80
building tree 26 of 80
building tree 27 of 80
building tree 28 of 80
building tree 29 of 80
building tree 30 of 80
building tree 31 of 80
building tree 32 of 80
building tree 33 of 80
building tree 34 of 80
building tree 35 of 80
building tree 36 of 80
building tree 37 of 80
building tree 38 of 80
building tree 39 of 80
building tree 40 of 80
building tree 41 of 80
building tree 42 of 80
building tree 43 of 80
building tree 44 of 80
building tree 45 of 80
building tree 46 of 80
building tree 47 of 80
building tree 48 of 80
building tree 49 of 80
building tree 50 of 80
building tree 51 of 80
building tree 52 of 80
building tree 53 of 80
building tree 54 of 80
building tree 55 of 80
building tree 56 of 80
building tree 57 of 80
building tree 58 of 80
building tree 59 of 80
building tree 60 of 80
building tree 61 of 80
building tree 62 of 80
building tree 63 of 80
building tree 64 of 80
building tree 65 of 80
building tree 66 of 80
building tr

[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 53.7min finished
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done  80 out of  80 | elapsed:    1.2s finished


Recall for  #Animals 0.664036817883
Threshold: 0.854020455376
Recall for  #Biz Durable Asset 0.052027027027
Threshold: 0.957610512277
Recall for  #Eco-friendly 0.788346613546
Threshold: 0.767409633486
Recall for  #Elderly 0.00162866449511
Threshold: 0.949444878294
Recall for  #Fabrics 0.421052631579
Threshold: 0.922523241543
Recall for  #Female Education 0.264516129032
Threshold: 0.926938545125
Recall for  #First Loan 0.0954274353877
Threshold: 0.899643336058
Recall for  #Health and Sanitation 0.462277091907
Threshold: 0.986815761279
Recall for  #Job Creator 0.00677200902935
Threshold: 0.909542715066
Recall for  #Orphan 0.0625
Threshold: 0.532634637233
Recall for  #Parent 0.00881735479356
Threshold: 0.999437498014
Recall for  #Refugee 0.0704225352113
Threshold: 0.809619724079
Recall for  #Repair Renew Replace 0.0117540687161
Threshold: 0.989470840254
Recall for  #Repeat Borrower 0.00678294573643
Threshold: 0.999798816978
Recall for  #Schooling 0.454659161976
Threshold: 0.882157576409
R

In [13]:
print(max_recall_score(labels_val, clf.predict_proba(combined_val)))
# 0.215005438828

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done  80 out of  80 | elapsed:    1.2s finished


Recall for  #Animals 0.664036817883
Threshold: 0.854020455376
Recall for  #Biz Durable Asset 0.052027027027
Threshold: 0.957610512277
Recall for  #Eco-friendly 0.788346613546
Threshold: 0.767409633486
Recall for  #Elderly 0.00162866449511
Threshold: 0.949444878294
Recall for  #Fabrics 0.421052631579
Threshold: 0.922523241543
Recall for  #Female Education 0.264516129032
Threshold: 0.926938545125
Recall for  #First Loan 0.0954274353877
Threshold: 0.899643336058
Recall for  #Health and Sanitation 0.462277091907
Threshold: 0.986815761279
Recall for  #Job Creator 0.00677200902935
Threshold: 0.909542715066
Recall for  #Orphan 0.0625
Threshold: 0.532634637233
Recall for  #Parent 0.00881735479356
Threshold: 0.999437498014
Recall for  #Refugee 0.0704225352113
Threshold: 0.809619724079
Recall for  #Repair Renew Replace 0.0117540687161
Threshold: 0.989470840254
Recall for  #Repeat Borrower 0.00678294573643
Threshold: 0.999798816978
Recall for  #Schooling 0.454659161976
Threshold: 0.882157576409
R

In [14]:
predictions_soft = clf.predict_proba(combined_test)

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  80 out of  80 | elapsed:    1.6s finished


In [13]:
print(predictions_soft)

[array([[ 0.07408244,  0.92591756],
       [ 0.15491762,  0.84508238],
       [ 0.96185556,  0.03814444],
       ..., 
       [ 0.13201383,  0.86798617],
       [ 0.13201383,  0.86798617],
       [ 0.09551404,  0.90448596]]), array([[ 0.63793071,  0.36206929],
       [ 0.59587106,  0.40412894],
       [ 0.92504742,  0.07495258],
       ..., 
       [ 0.79926695,  0.20073305],
       [ 0.79926695,  0.20073305],
       [ 0.75730473,  0.24269527]]), array([[ 0.99268756,  0.00731244],
       [ 0.99761637,  0.00238363],
       [ 0.95503489,  0.04496511],
       ..., 
       [ 0.99193874,  0.00806126],
       [ 0.99193874,  0.00806126],
       [ 0.99178471,  0.00821529]]), array([[ 0.78837459,  0.21162541],
       [ 0.66982862,  0.33017138],
       [ 0.78057393,  0.21942607],
       ..., 
       [ 0.77628483,  0.22371517],
       [ 0.77628483,  0.22371517],
       [ 0.80947638,  0.19052362]]), array([[  1.00000000e+00,   0.00000000e+00],
       [  9.99928333e-01,   7.16665967e-05],
       [ 

In [17]:
# 99.9 - 0.135340928608
# 99.5 - 0.187639548864
# 99 - 0.23032003206
# 98 - 0.285366691475
# 97 - 0.33520352665
# 96 - 0.40307436881
# 95 - 0.4439228259
# 94 - 0.488063204901
# 93 - 0.513854697429
# 92 - 0.541907597183
# 91 - 0.56787084216
# 90 - 0.585733096697
# 89 - 0.604912119998
# 88 - 0.622917501574
# 87 - 0.634968798305
# 86 - 0.652115417645
# 85 - 0.66617049293
# 84 - 0.680082441175
# 83 - 0.695168031144
# 82 - 0.712057021813
# 81 - 0.724509074254
# 80 - 0.736102364459
threshold = 0.96
real_threshold = math.sqrt(threshold*(1-threshold)/len(labels_test))*1.96+threshold
print("max recall score", max_recall_score(labels_test, predictions_soft, real_threshold))

Recall for  #Animals 0.753094059406
Threshold: 0.785655245695
Recall for  #Biz Durable Asset 0.163451232583
Threshold: 0.816439684928
Recall for  #Eco-friendly 0.795343800758
Threshold: 0.380035323244
Recall for  #Elderly 0.00482879719052
Threshold: 0.912540888232
Recall for  #Fabrics 0.658473479948
Threshold: 0.645381374726
Recall for  #Female Education 0.324528301887
Threshold: 0.929702142234
Recall for  #First Loan 0.0895140664962
Threshold: 0.722460268135
Recall for  #Health and Sanitation 0.919592298981
Threshold: 0.372989197749
Recall for  #Job Creator 0.012729844413
Threshold: 0.82675634534
Recall for  #Orphan 0.25
Threshold: 0.402826093032
Recall for  #Parent 0.050113081776
Threshold: 0.937206068292
Recall for  #Refugee 0.00851063829787
Threshold: 0.896162931656
Recall for  #Repair Renew Replace 0.0955631399317
Threshold: 0.880292568536
Recall for  #Repeat Borrower 0.251161519541
Threshold: 0.590667669341
Recall for  #Schooling 0.578947368421
Threshold: 0.581964935866
Recall fo

In [15]:
clf.fit(combined, labels)

building tree 6 of 80building tree 5 of 80building tree 1 of 80building tree 3 of 80building tree 4 of 80building tree 2 of 80building tree 7 of 80building tree 8 of 80







building tree 9 of 80
building tree 10 of 80
building tree 11 of 80
building tree 12 of 80
building tree 13 of 80
building tree 14 of 80
building tree 15 of 80
building tree 16 of 80
building tree 17 of 80
building tree 18 of 80
building tree 19 of 80
building tree 20 of 80
building tree 21 of 80
building tree 22 of 80
building tree 23 of 80


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 19.9min


building tree 24 of 80
building tree 25 of 80
building tree 26 of 80
building tree 27 of 80
building tree 28 of 80
building tree 29 of 80
building tree 30 of 80
building tree 31 of 80
building tree 32 of 80
building tree 33 of 80
building tree 34 of 80
building tree 35 of 80
building tree 36 of 80
building tree 37 of 80
building tree 38 of 80
building tree 39 of 80
building tree 40 of 80
building tree 41 of 80
building tree 42 of 80
building tree 43 of 80
building tree 44 of 80
building tree 45 of 80
building tree 46 of 80
building tree 47 of 80
building tree 48 of 80
building tree 49 of 80
building tree 50 of 80
building tree 51 of 80
building tree 52 of 80
building tree 53 of 80
building tree 54 of 80
building tree 55 of 80
building tree 56 of 80
building tree 57 of 80
building tree 58 of 80
building tree 59 of 80
building tree 60 of 80
building tree 61 of 80
building tree 62 of 80
building tree 63 of 80
building tree 64 of 80
building tree 65 of 80
building tree 66 of 80
building tr

[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 92.6min finished


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features=0.5,
            max_leaf_nodes=None, min_impurity_split=1e-06,
            min_samples_leaf=1, min_samples_split=16,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-1,
            oob_score=False, random_state=None, verbose=3,
            warm_start=False)

In [16]:
pickle.dump(clf, open('clf.pkl', 'wb'))